<a href="https://colab.research.google.com/github/seyonechithrananda/ncov-ligand-protein/blob/master/Final_90_10_Scaffold_GAT_ncov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now that we've ran all of our benchmarks and found the GAT-based classificaiton model trained on a Scaffold split to be the strongest, lets combine our SARS-COV-2 + SARS-COV-1 datasets (multi-task classifier) and train so we can utilize the model to screen repurposing drug assays for inhibition. We'll use a 90-10 train-test split so we can use as much data as possible for training the model (here, the test split only serves the purpose of verifing that the model performs alright).

In [1]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')


--2020-05-08 03:39:24--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  81.12M   175MB/s    in 0.5s    

2020-05-08 03:39:24 (175 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
  

In [2]:
!conda install -c dglteam dgl-cuda10.1

Solving environment: | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - dgl-cuda10.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |         openblas          46 KB
    certifi-2020.4.5.1         |           py37_0         155 KB
    decorator-4.4.2            |             py_0          14 KB
    dgl-cuda10.1-0.4.3post2    |           py37_0        11.2 MB  dglteam
    networkx-2.4               |             py_0         1.2 MB
    scipy-1.4.1                |   py37habc2bb6_0        14.6 MB
    ------------------------------------------------------------
                                           Total:        27.1 MB

The following NEW packages will be INSTALLED:

  blas               pkgs/main/linux-64::blas-1.0-openblas
  decorator          pkgs/main/noarch::deco

In [3]:
!conda install -c dglteam dgllife


Solving environment: - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - dgllife


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dgllife-0.2.1              |           py37_0         132 KB  dglteam
    joblib-0.14.1              |             py_0         201 KB
    scikit-learn-0.22.1        |   py37h22eb022_0         5.3 MB
    ------------------------------------------------------------
                                           Total:         5.6 MB

The following NEW packages will be INSTALLED:

  dgllife            dglteam/linux-64::dgllife-0.2.1-py37_0
  joblib             pkgs/main/noarch::joblib-0.14.1-py_0
  scikit-learn       pkgs/main/linux-64::scikit-learn-0.22.1-py37h22eb022_0


Proceed ([y]/n)? y


scikit-learn-0.22.1  | 5.3 MB    | : 100% 1.0/1 [00:00<00:00,  1.48s/it]         

In [4]:
!conda install pandas

Solving environment: / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |           py37_0         2.8 MB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    pandas-1.0.3               |   py37h0573a6f_0         8.6 MB
    ------------------------------------------------------------
                                           Total:        13.9 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> pkgs/main::ca-certificates-2020.1.1-0
  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> pkgs/main::conda-4.8.3-py37_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> pkgs/main::openssl-1.1

In [0]:
import os 
import sys 
import pandas as pd

# train --> balanced dataset
dataset_file = '/content/drive/My Drive/Project De Novo/AID1706_binarized_sars_mpro_xchem_full_eval_actives_12k_samples.csv - AID1706_binarized_sars.csv'
dataset = pd.read_csv(dataset_file)


In [6]:
dataset.head

<bound method NDFrame.head of                                                   smiles  activity
0      C1CC(C1)C(=O)NC2=CC=C(C=C2)N(C(C3=CC(=CC=C3)F)...         1
1      CC(C)(C)C1=CC=C(C=C1)N(C(C2=CN=NC=C2)C(=O)NC(C...         1
2      CC(C)(C)NC(=O)C(C1=CSC=C1)N(C2=CC=C(C=C2)N)C(=...         1
3      CC(C)C(=O)NC1=CC=C(C=C1)N(C(C2=CSC=C2)C(=O)NC(...         1
4      CC(C)C(=O)NC1=CC=C(C=C1)N(CC2=CSC=C2)C(=O)CN3C...         1
...                                                  ...       ...
12874                   CC(C)C=1C=CC(NC(=O)N2CCOCC2)=CC1         0
12875                        CN(CC(=O)O)C(=O)C=1C=CC=CN1         0
12876                  CN1CCN(CC1)C(=O)C=2C=CC(F)=C(F)C2         0
12877                      FC=1C=CC=C(F)C1C(=O)N2CCCCCC2         0
12878                             FC=1C=CC=NC1NCC2CCOCC2         0

[12879 rows x 2 columns]>

In [7]:
dataset.tail

<bound method NDFrame.tail of                                                   smiles  activity
0      C1CC(C1)C(=O)NC2=CC=C(C=C2)N(C(C3=CC(=CC=C3)F)...         1
1      CC(C)(C)C1=CC=C(C=C1)N(C(C2=CN=NC=C2)C(=O)NC(C...         1
2      CC(C)(C)NC(=O)C(C1=CSC=C1)N(C2=CC=C(C=C2)N)C(=...         1
3      CC(C)C(=O)NC1=CC=C(C=C1)N(C(C2=CSC=C2)C(=O)NC(...         1
4      CC(C)C(=O)NC1=CC=C(C=C1)N(CC2=CSC=C2)C(=O)CN3C...         1
...                                                  ...       ...
12874                   CC(C)C=1C=CC(NC(=O)N2CCOCC2)=CC1         0
12875                        CN(CC(=O)O)C(=O)C=1C=CC=CN1         0
12876                  CN1CCN(CC1)C(=O)C=2C=CC(F)=C(F)C2         0
12877                      FC=1C=CC=C(F)C1C(=O)N2CCCCCC2         0
12878                             FC=1C=CC=NC1NCC2CCOCC2         0

[12879 rows x 2 columns]>

In [8]:
from dgllife.data import MoleculeCSVDataset
from dgllife.data.csv_dataset import *
from dgllife.utils.featurizers import *
from dgllife.utils.mol_to_graph import *

# featurize bigraph/molecular graph set for train (SARS-COV-1) set
csv_dataset = MoleculeCSVDataset(dataset, smiles_to_graph=smiles_to_bigraph, node_featurizer=CanonicalAtomFeaturizer(),
                               edge_featurizer=CanonicalBondFeaturizer(), smiles_column='smiles', cache_file_path='/content/drive/My Drive/Project De Novo/graph_featurizer/sars-mpro-xchem.bin', task_names=['activity'])

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Processing dgl graphs from scratch...
Processing molecule 1000/12879
Processing molecule 2000/12879
Processing molecule 3000/12879
Processing molecule 4000/12879
Processing molecule 5000/12879
Processing molecule 6000/12879
Processing molecule 7000/12879
Processing molecule 8000/12879
Processing molecule 9000/12879
Processing molecule 10000/12879
Processing molecule 11000/12879
Processing molecule 12000/12879


In [9]:
from google.colab import files
files.upload()

Saving utils.py to utils.py


{'utils.py': b'import dgl\nimport numpy as np\nimport random\nimport torch\n\nfrom dgllife.utils.featurizers import one_hot_encoding\nfrom dgllife.utils.splitters import RandomSplitter\n\ndef set_random_seed(seed=0):\n    """Set random seed.\n    Parameters\n    ----------\n    seed : int\n        Random seed to use\n    """\n    random.seed(seed)\n    np.random.seed(seed)\n    torch.manual_seed(seed)\n    if torch.cuda.is_available():\n        torch.cuda.manual_seed(seed)\n\n\ndef load_dataset_for_classification(args):\n    """Load dataset for classification tasks.\n    Parameters\n    ----------\n    args : dict\n        Configurations.\n    Returns\n    -------\n    dataset\n        The whole dataset.\n    train_set\n        Subset for training.\n    val_set\n        Subset for validation.\n    test_set\n        Subset for test.\n    """\n    assert args[\'dataset\'] in [\'Tox21\']\n    if args[\'dataset\'] == \'Tox21\':\n        from dgllife.data import Tox21\n        dataset = Tox

In [0]:
args = {
    'random_seed': 2,
    'batch_size': 128,
    'lr': 1e-3,
    'num_epochs': 250,
    'node_data_field': 'h',
    'frac_train': 0.8,
    'frac_val': 0.1,
    'frac_test': 0.1,
    'in_feats': 74,
    'gat_hidden_feats': [32, 32],
    'classifier_hidden_feats': 64,
    'num_heads': [4, 4],
    'patience': 50,
    'smiles_to_graph': smiles_to_bigraph,
    'node_featurizer': CanonicalAtomFeaturizer(),
    'metric_name': 'roc_auc_score',
    'model': 'GAT'
}


In [0]:
import numpy as np
import torch
import utils

from dgllife.model import load_pretrained
from dgllife.utils import EarlyStopping, Meter
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader


from utils import set_random_seed, load_dataset_for_classification, collate_molgraphs, load_model

from dgllife.model import GATPredictor

args['device'] = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_random_seed(args['random_seed'])



In [13]:
from dgllife.utils.splitters import ScaffoldSplitter

train_scaffold_set, val_set, test_scaffold_set = ScaffoldSplitter.train_val_test_split(csv_dataset, frac_train=0.9, frac_val=0.1,frac_test=0.0)

Start initializing RDKit molecule instances...
Creating RDKit molecule instance 1000/12879
Creating RDKit molecule instance 2000/12879
Creating RDKit molecule instance 3000/12879
Creating RDKit molecule instance 4000/12879
Creating RDKit molecule instance 5000/12879
Creating RDKit molecule instance 6000/12879
Creating RDKit molecule instance 7000/12879
Creating RDKit molecule instance 8000/12879
Creating RDKit molecule instance 9000/12879
Creating RDKit molecule instance 10000/12879
Creating RDKit molecule instance 11000/12879
Creating RDKit molecule instance 12000/12879
Start computing Bemis-Murcko scaffolds.
Computing Bemis-Murcko for compound 1000/12879
Computing Bemis-Murcko for compound 2000/12879
Computing Bemis-Murcko for compound 3000/12879
Computing Bemis-Murcko for compound 4000/12879
Computing Bemis-Murcko for compound 5000/12879
Computing Bemis-Murcko for compound 6000/12879
Computing Bemis-Murcko for compound 7000/12879
Computing Bemis-Murcko for compound 8000/12879
Comput

In [17]:
print (len(csv_dataset))
print(len(train_scaffold_set))
print(len(test_scaffold_set))
print (len(val_set))
print (len(test_scaffold_set))
print(csv_dataset[1])

12879
11591
0
1288
0
('CC(C)(C)C1=CC=C(C=C1)N(C(C2=CN=NC=C2)C(=O)NC(C)(C)C)C(=O)C3=CC=CO3', DGLGraph(num_nodes=32, num_edges=68,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}), tensor([1.]), tensor([1.]))


In [18]:
print(type(csv_dataset))
print(type(train_scaffold_set))

<class 'dgllife.data.csv_dataset.MoleculeCSVDataset'>
<class 'dgl.data.utils.Subset'>


In [0]:
train_loader = DataLoader(train_scaffold_set,  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)
val_loader = DataLoader(val_set,  batch_size=args['batch_size'],
                              collate_fn=collate_molgraphs)

#test_loader = DataLoader(test_set,  batch_size=args['batch_size'],
#                          collate_fn=collate_molgraphs)

In [26]:
print(len(train_loader))
print(len(val_loader))

91
11


In [43]:
args['n_tasks'] = 1
model = GATPredictor(in_feats=args['node_featurizer'].feat_size('h'),
                             hidden_feats=args['gat_hidden_feats'],
                             num_heads=args['num_heads'],
                             classifier_hidden_feats=args['classifier_hidden_feats'],
                             n_tasks=args['n_tasks'])

import dgl.backend as F



train_num_pos = F.sum(csv_dataset.labels, dim=0)
train_num_indices = F.sum(csv_dataset.mask, dim=0)
train_task_pos_weights = (train_num_indices - train_num_pos) / train_num_pos

loss_criterion = BCEWithLogitsLoss(pos_weight=train_task_pos_weights.to(args['device']),
                                    reduction='none')

optimizer = Adam(model.parameters(), lr=args['lr'])
stopper = EarlyStopping(patience=args['patience'], mode='higher', filename='/content/drive/My Drive/Project De Novo/GAT/train_scaffold_combined_data_250.pth')
model.to(args['device'])


GATPredictor(
  (gnn): GAT(
    (gnn_layers): ModuleList(
      (0): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=74, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=74, out_features=128, bias=False)
        )
      )
      (1): GATLayer(
        (gat_conv): GATConv(
          (fc): Linear(in_features=128, out_features=128, bias=False)
          (feat_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (leaky_relu): LeakyReLU(negative_slope=0.2)
          (res_fc): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
  )
  (readout): WeightedSumAndMax(
    (weight_and_sum): WeightAndSum(
      (atom_weighting): Sequential(
        (0): Linear(in_features=32, out_features=1, bias=True)
        (1): Sigmoid()


In [44]:
print(train_task_pos_weights)

tensor([23.5782])


In [0]:
def predict(args, model, bg):
    node_feats = bg.ndata.pop(args['node_data_field']).to(args['device'])
    if args.get('edge_featurizer', None) is not None:
        edge_feats = bg.edata.pop(args['edge_data_field']).to(args['device'])
        return model(bg, node_feats, edge_feats)
    else:
        return model(bg, node_feats)

def run_a_train_epoch(args, epoch, model, data_loader, loss_criterion, optimizer):
    model.train()
    train_meter = Meter()
    for batch_id, batch_data in enumerate(data_loader):
        smiles, bg, labels, masks = batch_data
        labels, masks = labels.to(args['device']), masks.to(args['device'])
        print(bg)
        logits = predict(args, model, bg)
        # Mask non-existing labels
        loss = (loss_criterion(logits, labels) * (masks != 0).float()).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('epoch {:d}/{:d}, batch {:d}/{:d}, loss {:.4f}'.format(
            epoch + 1, args['num_epochs'], batch_id + 1, len(data_loader), loss.item()))
        train_meter.update(logits, labels, masks)
    train_score = np.mean(train_meter.compute_metric(args['metric_name']))
    print('epoch {:d}/{:d}, training {} {:.4f}'.format(
        epoch + 1, args['num_epochs'], args['metric_name'], train_score))

def run_an_eval_epoch(args, model, data_loader):
    model.eval()
    eval_meter = Meter()
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            smiles, bg, labels, masks = batch_data
            labels = labels.to(args['device'])
            logits = predict(args, model, bg)
            eval_meter.update(logits, labels, masks)
    return np.mean(eval_meter.compute_metric(args['metric_name']))


In [46]:
for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch(args, epoch, model, train_loader, loss_criterion, optimizer)

        # Validation and early stop
        val_score = run_an_eval_epoch(args, model, val_loader)
        early_stop = stopper.step(val_score, model)
        print('epoch {:d}/{:d}, validation {} {:.4f}, best validation {} {:.4f}'.format(
            epoch + 1, args['num_epochs'], args['metric_name'],
            val_score, args['metric_name'], stopper.best_score))
        if early_stop:
            break

Streaming output truncated to the last 5000 lines.
epoch 43/250, batch 35/91, loss 0.0814
DGLGraph(num_nodes=3377, num_edges=7328,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 43/250, batch 36/91, loss 0.0848
DGLGraph(num_nodes=3305, num_edges=7154,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 43/250, batch 37/91, loss 0.0598
DGLGraph(num_nodes=3321, num_edges=7218,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 43/250, batch 38/91, loss 0.0666
DGLGraph(num_nodes=3371, num_edges=7354,
         ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
         edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})
epoch 43/250, batch 39/91, loss 0.1299
DGLGraph(num_nodes

In [0]:
# no test class, in order to save data. best validation is treated as metric. (similar to how k-fold cross validation is done with two tuples)
stopper.load_checkpoint(model)
test_score = run_an_eval_epoch(args, model, test_loader)
print('test {} {:.4f}'.format(args['metric_name'], test_score))
